## 결측값을 예측모형으로 예측치 넣기 ##

* sklearn(사이킷런) 사용   
* ind_cols: 예측하고 싶은 값에 영향을 주는 요소 중 서로 독립적인 칼럼명만 넣어둔다.
* train, test, predict 데이터 x 값에 바로 넣기 편하다.


In [103]:
import pandas as pd 
import sklearn

df_ms=pd.read_csv('./df_funnel.csv')
print(df_ms.isnull())

df_msl = df_ms.dropna()

# set independent cols  <-- 서로 영향 없는 칼럼만 고르기

ind_cols = ['actiontype', 'ismydoc', 'ext', 'screen']

#split ind, tar --> 학습시키고 싶은 칼럼과 인풋 칼럼 나누기
x = df_msl[ind_cols]
y = df_msl[['documentposition']]


        Unnamed: 0  actiontype  ismydoc    ext  sessionid  documentposition  \
0            False       False    False  False      False             False   
1            False       False    False  False      False             False   
2            False       False    False  False      False             False   
3            False       False    False  False      False             False   
4            False       False    False  False      False             False   
...            ...         ...      ...    ...        ...               ...   
301856       False       False    False  False      False              True   
301857       False       False    False  False      False              True   
301858       False       False    False  False      False              True   
301859       False       False    False  False      False              True   
301860       False       False    False  False      False             False   

        datetime  screen  
0          False   False

항목값 전처리 (Label Encoder)
* 인코딩은 카테고리 값이나 텍스트 정보를 처리가 쉬운 정수로 변환하는 과정
* Label encoder는 독립 변수가 아닌 종속 변수 (라벨)에 대해 사용
* 문자열이나 정수로 된 라벨 값을 0~K-1까지의 정수로 변환 

In [104]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

X = x.apply(lambda x: encoder.fit_transform(x))
y = y.apply(lambda y: encoder.fit_transform(y))
# .inverse_transform()를 이용해서 역방향도 가능하다.

훈련데이터와 평가데이터 추출
* train_test_split는 데이터의 표본을 무작위로 추출해 훈련 데이터와 평가 데이터로 나눔
* random_state 속성값을 이용해서 적절히 섞은 곳에서 추출 가능

In [105]:
from sklearn.model_selection import train_test_split

# X: df_ms_ind / y: df_ms_tar
X_train, X_test, y_train, y_test = ?(X, y, random_state=10)

학습모형 만들기: KNN 알고리즘
* K-최근접 이웃 알고리즘 (KNN)은 분류나 회귀에서 사용되는 ML 기법
* 특정한 공간 내에 K개의 가장 가까운 훈련 데이터를 사용하여 해당 값을 예측하는 모델
* KNeighborsClassifier 함수를 사용하여 ML 모델을 정의 (n_neighbors는 K개의 갯수를 조절)
* Fit 함수를 사용하여 학습에 사용할 데이터를 명시
* Score 함수를 사용하여 성능 평가 !


In [106]:
from sklearn.neighbors import KNeighborsClassifier

knn = ? # ML 모델 정의
knn.? (X_train, y_train) # 학습에 사용할 데이터 명시
print(knn.score(X_test, y_test) # Score 함수를 사용하여 성능 평가 !


<ipython-input-106-1cc272030978>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  knn = KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train)


학습모형 평가하기 - Score
* 학습을 통해 만들어진 예측 모형에 평가 데이터를 넣어 실제값과 얼마나 유사하게 예측하는지 평가한다.
* 즉, X를 넣어 나온 predicted Y가 넣어준 Y와 맞는지 확인
* 즉, 스코어는 해당 모델의 예측 정확도를 확인할 수 있음

In [107]:
print(knn.score(X_test, y_test))

0.9998012165546853


결측값에 예측치 채워 넣기

In [114]:
# 인코딩에서 숫자로 변환되었던 항목값 
class_cd = pd.Series(encoder.classes_).to_dict()

print(encoder.classes_)

# 결측치가 하나 이상 있는 Case만 선택
df_ms_only = df_ms[df_ms.isna().any(axis=1)]
# 결측치가 없는 Case만 선택
df_ms_dropna = df_ms.dropna()

# 평가에 필요한 칼럼만 뽑으면서, 동시에 인코딩 진행
df_ms_only[ind_cols].apply(lambda x: encoder.fit_transform(x))
# 학습모형에 집어 넣어 값 예측하기
knn.predict(df_ms_only[ind_cols].apply(lambda x: encoder.fit_transform(x)))
# 예측값 저장하기
fill_na_values = knn.predict(df_ms_only[ind_cols].apply(lambda x: encoder.fit_transform(x)))

# 결측치가 있는 곳에 예측값 집어 넣기
df_ms_only['documentposition'] = fill_na_values

# 인코딩된 칼럼값을 되돌리기
df_ms_only['documentposition'] = df_ms_only['documentposition'].replace(class_cd)

#원래 데이터에 합치기
df_ms_final = pd.concat([df_ms_dropna, df_ms_only], axis=0)

df_ms_final.isnull()

['InProduct_Mob' 'Main' 'Pub_Dir']


<ipython-input-114-7d58c35bd3e4>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ms_only['documentposition'] = fill_na_values
<ipython-input-114-7d58c35bd3e4>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ms_only['documentposition'] = df_ms_only['documentposition'].replace(class_cd)


,Unnamed: 0,actiontype,ismydoc,ext,sessionid,documentposition,datetime,screen
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
301855,False,False,False,False,False,False,False,False
301856,False,False,False,False,False,False,False,False
301857,False,False,False,False,False,False,False,False
301858,False,False,False,False,False,False,False,False
